In [1]:
import numpy as np
import time
import sklearn
from scipy.linalg import khatri_rao
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [3]:
train_data = np.loadtxt('public_trn.txt')
train_questions = train_data[:, :32]
train_answers = train_data[:, 32:]

In [4]:
test_data = np.loadtxt('public_tst.txt')
test_questions = test_data[:, :32]
test_answers = test_data[:, 32:]

In [5]:
print(train_questions.shape)
print(train_answers.shape)
print(test_questions.shape)
print(test_answers.shape)

(40000, 32)
(40000, 2)
(10000, 32)
(10000, 2)


In [6]:
def my_map(X):
  transformed_X = np.cumprod(1-2*np.flip(X, axis=1), axis=1)
  features = np.hstack((X, transformed_X))
  return features

In [7]:
train_features = my_map(train_questions)
test_features = my_map(test_questions)

In [8]:
print(train_features.shape)
print(test_features.shape)

(40000, 64)
(10000, 64)


In [11]:
def my_fit(X_train, y0_train, y1_train):
    clf0 = LinearSVC(loss='hinge', C=1)
    clf1 = LinearSVC(loss='hinge', C=1)

    start1 = time.time()
    clf0.fit(X_train, y0_train)
    w0 = clf0.coef_[0]
    b0 = clf0.intercept_[0]
    end1 = time.time()

    start2 = time.time()
    clf1.fit(X_train, y1_train)
    w1 = clf1.coef_[0]
    b1 = clf1.intercept_[0]
    end2 = time.time()

    training_time1 = end1 - start1
    training_time2 = end2 - start2

    return w0, b0, w1, b1, training_time1, training_time2

In [12]:
w0, b0, w1, b1, train_time1, train_time2 = my_fit(train_features, train_answers[:, 0], train_answers[:, 1])

y0_train_pred = (np.dot(train_features, w0) + b0 > 0).astype(int)
y1_train_pred = (np.dot(train_features, w1) + b1 > 0).astype(int)

train_accuracy_0 = accuracy_score(train_answers[:, 0], y0_train_pred)
train_accuracy_1 = accuracy_score(train_answers[:, 1], y1_train_pred)

print(f"Train Accuracy for Response0: {train_accuracy_0:.4f}")
print(f"Train Accuracy for Response1: {train_accuracy_1:.4f}")

y0_test_pred = (np.dot(test_features, w0) + b0 > 0).astype(int)
y1_test_pred = (np.dot(test_features, w1) + b1 > 0).astype(int)

test_accuracy_0 = accuracy_score(test_answers[:, 0], y0_test_pred)
test_accuracy_1 = accuracy_score(test_answers[:, 1], y1_test_pred)

print(f"Test Accuracy for Response0: {test_accuracy_0:.4f}")
print(f"Test Accuracy for Response1: {test_accuracy_1:.4f}")
print(f"Training time: {train_time1:.4f}")
print(f"Training time: {train_time2:.4f}")

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Train Accuracy for Response0: 0.9814
Train Accuracy for Response1: 0.9964
Test Accuracy for Response0: 0.9838
Test Accuracy for Response1: 0.9949
Training time: 0.4100
Training time: 0.2757


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
